# SageMaker Endpoint - Invoke with boto3

## 0. 배포한 SageMaker Endpoint Name 불러오기

In [ ]:
# SageMaker Endpoint 와 통신하기 위해서 저장했던 변수를 불러옵니다.
%store -r endpoint_name
print(f"SageMaker EndpointName: {endpoint_name}")
print()

## 1. boto3 를 이용해서 SageMaker Endpoint 에 배포된 모델 호출

In [ ]:
import json
import boto3
import datetime

region = boto3.Session().region_name
sagemaker_runtime = boto3.client(
    "sagemaker-runtime",
    region_name=region
)

In [ ]:
client = boto3.client("sagemaker-runtime", region_name='us-west-2')

system ="You are a helpful assistant" 
text = "Hello! Who are you?"
prompt=f"""
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

{system}|eot_id|><|start_header_id|>user<|end_header_id|>

{text}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""

payload = {
    "inputs": prompt,
    "parameters": {
        "max_new_tokens": 128,
        "top_p": 0.9,
        "temperature": 0.9,
        # "return_full_text": False
    }
}

time_to_first_token = -1
start_time = datetime.datetime.now()

response = client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="application/json",
    Body=json.dumps(payload),
)

# 응답 처리
end_time = datetime.datetime.now()
response_body = json.loads(response['Body'].read().decode())

# 응답 시간 계산
total_time = (end_time - start_time).total_seconds()

# 결과 출력
print(f"# System\n- {system}\n")
print(f"# Text\n- {text}\n")
print(f"# Prompt\n- {prompt}")
print(f"# Response\n- {response_body['generated_text']}\n")
print(f"# Total time\n- {total_time:.2f} seconds\n")

## 2. SageMaker Endpoint 삭제

In [ ]:
# predictor.delete_predictor()